In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [2]:
spark = SparkSession.builder \
    .appName("ner")\
    .master("local[1]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [3]:
import time

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetectorModel()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = RegexTokenizer()\
  .setInputCols(["document"])\
  .setOutputCol("token")

extractor = EntityExtractor()\
  .setEntitiesPath("entities.txt")\
  .setInputCols(["token", "sentence"])\
  .setOutputCol("entites")

finisher = Finisher() \
    .setInputCols(["entites"]) \
    .setIncludeKeys(True)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    sentenceDetector,
    tokenizer,
    extractor,
    finisher
  ])


In [4]:
#Load the input data to be annotated
data = spark. \
        read. \
        parquet("../../../src/test/resources/sentiment.parquet"). \
        limit(1000)
data.cache()
data.count()
data.show()

+------+---------+--------------------+
|itemid|sentiment|                text|
+------+---------+--------------------+
|     1|        0|                 ...|
|     2|        0|                 ...|
|     3|        1|              omg...|
|     4|        0|          .. Omga...|
|     5|        0|         i think ...|
|     6|        0|         or i jus...|
|     7|        1|       Juuuuuuuuu...|
|     8|        0|       Sunny Agai...|
|     9|        1|      handed in m...|
|    10|        1|      hmmmm.... i...|
|    11|        0|      I must thin...|
|    12|        1|      thanks to a...|
|    13|        0|      this weeken...|
|    14|        0|     jb isnt show...|
|    15|        0|     ok thats it ...|
|    16|        0|    &lt;-------- ...|
|    17|        0|    awhhe man.......|
|    18|        1|    Feeling stran...|
|    19|        0|    HUGE roll of ...|
|    20|        0|    I just cut my...|
+------+---------+--------------------+
only showing top 20 rows



In [5]:
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")

Start fitting
Fitting is ended


In [6]:
extracted = model.transform(data)
extracted.show()

+------+---------+--------------------+-------------------+
|itemid|sentiment|                text|   finished_entites|
+------+---------+--------------------+-------------------+
|     1|        0|                 ...|                   |
|     2|        0|                 ...|                   |
|     3|        1|              omg...|                   |
|     4|        0|          .. Omga...|                   |
|     5|        0|         i think ...|    result->i think|
|     6|        0|         or i jus...|                   |
|     7|        1|       Juuuuuuuuu...|                   |
|     8|        0|       Sunny Agai...|                   |
|     9|        1|      handed in m...|                   |
|    10|        1|      hmmmm.... i...|                   |
|    11|        0|      I must thin...|                   |
|    12|        1|      thanks to a...|                   |
|    13|        0|      this weeken...|                   |
|    14|        0|     jb isnt show...| 

In [7]:
pipeline.write().overwrite().save("./ner_pipeline")
model.write().overwrite().save("./ner_model")

In [8]:
from pyspark.ml import PipelineModel, Pipeline

Pipeline.read().load("./ner_pipeline")
sameModel = PipelineModel.read().load("./ner_model")